In [15]:
from azureml.core import Workspace, Experiment

# ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: myvscodews
Azure region: centralus
Subscription id: aef81eda-3a04-489e-81ab-1ffaed381a6a
Resource group: ws07131505-rg1


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Use https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.computetarget?view=azure-ml-py as a reference
#
cluster_name = "CheapCPUCluster"

# See if it already exists
try:
    compute_cluster = ComputeTarget(ws, cluster_name)
    # Found it - use it below
except:
    # Did not find the compute target - will need to create one
    # Specify the compute cluster configuration first
    # See https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target
    # and after clicking on Dv2
    # see https://docs.microsoft.com/en-us/azure/machine-learning/concept-compute-target

    # CPU cluster instance
    # cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_v2',
    #                                                         max_nodes=4,
    #                                                         description='Compute Cluster created programatically')

    cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_DS2_v2',
                                                            max_nodes=1,
                                                            description='Cheap Compute Cluster created programatically')


    # GPU cluster instance
    # cluster_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_NC6',
    #                                                         max_nodes=4,
    #                                                         description='Compute Cluster created programatically')


    # Next, create the cluster
    compute_cluster = ComputeTarget.create(ws, cluster_name, cluster_config)

# We have a compute cluster - either newly created - or created earlier

# We may wait for the create operation to complete
compute_cluster.wait_for_completion(show_output=True)


Creating......
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.core import Experiment

In [7]:
from azureml.core import ScriptRunConfig

In [9]:
from azureml.core.environment import Environment

In [10]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-VowpalWabbit-8.8.0
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.19.0
  - azureml-defaults==1.19.0
  - azureml-dataset-runtime[fuse,pandas]
name: azureml_769be4b756b756954fa484d1287d5153

Name AzureML-TensorFlow-2.3-CPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip=20.2.4
- pip:
  - azureml-core==1.32.0
  - azureml-defaults==1.32.0
  - azureml-telemetry==1.32.0
  - azureml-train-restclients-hyperdrive==1.32.0
  - azureml-train-core==1.32.0
  - tensorflow==2.3.0
  - cmake==3.18.2
  - horovod==0.21.3
name: azureml_d6f7dec5977790b91df09029a7e8f48a

Name AzureML-Triton
packages channels:
- conda-forge
dependencies:
- python=3.7.9
- pip:
  - azureml-core==1.32.0
  - azureml-defaults[async]
  - azureml-contrib-services==1.32.0
  - numpy
  - inference-schema[numpy-support]
  - grpcio-tools
  - geventhttpclient
  - https://developer.download.nvidia.com/compute/redist/tritonclient/tritonclient-2.7.0-py3-none-manylinux

In [47]:

# Specify parameter sampler
# ps = ### YOUR CODE HERE ###
# A lot of rederence information may be found at 
# "Hyperparameter tuning a model with Azure Machine Learning" - https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

#import normal and choice as they are used below in the parameter sampling
from azureml.train.hyperdrive.parameter_expressions import normal, choice

ps = RandomParameterSampling({
    "--C": choice(0.001, 0.01, 0.1, 1, 10),
    "--max_iter": choice(100, 200, 300, 400, 500),
})
# Specify a Policy
#policy = ### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, delay_evaluation=5)

# if "training" not in os.listdir():
#     os.mkdir("./training")


# Create a SKLearn estimator for use with train.py
# est = ### YOUR CODE HERE ###
# est = SKLearn(
#     source_directory=".", # No sure about where this belongs
#     compute_target= compute_cluster, # or is it 'local'
#     entry_script="train.py")

curated_env_name = 'AzureML-Tutorial'
curated_env = Environment.get(workspace=ws, name=curated_env_name)

run_config = ScriptRunConfig(
    source_directory=".",
    script="mybrain.py",
    arguments=['--C', 1.0, '--max_iter', 100],
    compute_target=compute_cluster,
    environment=curated_env,
)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
# hyperdrive_config = ### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    policy=policy,
    run_config=run_config,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=1,
)

In [48]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
# from azureml.core import Experiment 


In [49]:
hdr = exp.submit(config=hyperdrive_config)

In [50]:
RunDetails(hdr).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [51]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###